# Chapter 5: Functions and Modular Programming

Functions are the primary mechanism for code organization and reuse in Python. They allow you to encapsulate logic, reduce redundancy, and create abstractions that make complex systems manageable. Mastering functions—understanding not just their syntax but their execution model, scope rules, and design patterns—is what transforms a scripter into a software engineer.

In this chapter, we will explore how to define robust, well-documented functions using modern Python conventions, understand Python's unique scoping rules (the LEGB rule), leverage functions as first-class objects, and organize code into modules and packages following industry standards.

## 5.1 Defining Functions

A function is defined using the `def` keyword, followed by a name, parameters in parentheses, and a colon. The body is indented.

### Basic Structure and Type Hints

Modern Python code uses **type hints** to indicate expected parameter types and return types. While not enforced at runtime (Python remains dynamically typed), type hints enable static analysis, IDE autocomplete, and documentation.

```python
def calculate_area(length: float, width: float) -> float:
    """Calculate the area of a rectangle."""
    return length * width

# Function with no return value (None)
def print_greeting(name: str) -> None:
    """Print a greeting message."""
    print(f"Hello, {name}!")

# Function with optional return
def safe_divide(a: float, b: float) -> float | None:
    """Divide two numbers, returning None if division by zero."""
    if b == 0:
        return None
    return a / b
```

**Industry Standards:**
*   Use descriptive verb-noun names: `get_user_data()`, not `data()`.
*   All functions should have docstrings (see section 5.7).
*   Always use type hints in production code.
*   Keep functions focused: one function should do one thing (Single Responsibility Principle).

### The `return` Statement

The `return` statement exits a function and passes a value back to the caller. Without an explicit `return`, functions return `None`.

```python
def multiply(a: int, b: int) -> int:
    product = a * b
    return product

# Multiple return values (actually returns a tuple)
def get_min_max(numbers: list[int]) -> tuple[int, int]:
    """Return both minimum and maximum values."""
    return min(numbers), max(numbers)

# Unpacking the return value
minimum, maximum = get_min_max([3, 1, 4, 1, 5])
```

**Early Returns:** Returning early can reduce nesting and improve readability (guard clauses).

```python
def process_user(user: dict[str, any] | None) -> dict[str, str]:
    """Process user data with validation."""
    # Guard clause - exit early on invalid input
    if user is None:
        return {"error": "No user provided"}
    
    if not user.get("is_active"):
        return {"error": "User is inactive"}
    
    # Main logic only executes for valid input
    return {"status": "processed", "name": user["name"]}
```

## 5.2 Parameters and Arguments

Understanding the distinction between **parameters** (variables in the definition) and **arguments** (values passed during the call) is fundamental.

### Positional and Keyword Arguments

```python
def create_user(username: str, email: str, age: int, is_admin: bool = False) -> dict:
    return {
        "username": username,
        "email": email,
        "age": age,
        "is_admin": is_admin
    }

# Positional arguments (order matters)
user1 = create_user("alice", "alice@example.com", 30)

# Keyword arguments (order doesn't matter, readable)
user2 = create_user(
    email="bob@example.com", 
    username="bob", 
    age=25
)

# Mixed (positional must come before keyword)
user3 = create_user("charlie", "charlie@example.com", age=35, is_admin=True)
```

**Industry Guideline:** For functions with more than 2-3 parameters, prefer keyword arguments at the call site for readability ("self-documenting code").

### Default Parameter Values

Default values allow optional parameters. However, **never use mutable default values**—this is one of Python's most infamous gotchas.

```python
# WRONG: Mutable default argument
def append_item_wrong(item: int, target_list: list[int] = []) -> list[int]:
    """DANGER: The default list is shared between calls!"""
    target_list.append(item)
    return target_list

print(append_item_wrong(1))  # [1]
print(append_item_wrong(2))  # [1, 2] - Surprising! The list persisted!

# CORRECT: Use None as sentinel
def append_item_correct(item: int, target_list: list[int] | None = None) -> list[int]:
    """Safely append to a list, creating new if none provided."""
    if target_list is None:
        target_list = []
    target_list.append(item)
    return target_list

print(append_item_correct(1))  # [1]
print(append_item_correct(2))  # [2] - Correct behavior
```

### Variable-Length Arguments: `*args` and `**kwargs`

When you need to accept arbitrary numbers of arguments, Python provides special syntax.

**`*args` (Arbitrary Positional Arguments):**
Collects extra positional arguments into a **tuple**.

```python
def calculate_sum(*numbers: int | float) -> float:
    """Sum any number of numeric arguments."""
    total: float = 0.0
    for num in numbers:
        total += num
    return total

# Usage
result = calculate_sum(1, 2, 3, 4, 5)  # numbers is (1, 2, 3, 4, 5)
```

**Unpacking with `*`:**
You can also use `*` when calling functions to unpack iterables into positional arguments.

```python
values = [1, 2, 3, 4]
result = calculate_sum(*values)  # Equivalent to calculate_sum(1, 2, 3, 4)
```

**`**kwargs` (Arbitrary Keyword Arguments):**
Collects extra keyword arguments into a **dictionary**.

```python
def build_profile(name: str, **attributes: any) -> dict[str, any]:
    """Build a user profile with required name and optional attributes."""
    profile = {"name": name}
    profile.update(attributes)
    return profile

# Usage
user = build_profile(
    "Alice",
    age=30,
    city="NYC",
    hobbies=["reading", "coding"]
)
# attributes is {'age': 30, 'city': 'NYC', 'hobbies': ['reading', 'coding']}
```

**Keyword-Only Arguments (Python 3+):**
Force callers to use keyword syntax for certain parameters, improving clarity and allowing parameter reordering.

```python
def configure_database(
    host: str,
    port: int,
    *,
    ssl_enabled: bool = True,
    timeout: int = 30
) -> dict[str, any]:
    """
    Configure database connection.
    Parameters after * must be specified as keywords.
    """
    return {
        "host": host,
        "port": port,
        "ssl_enabled": ssl_enabled,
        "timeout": timeout
    }

# Valid
config = configure_database("localhost", 5432, ssl_enabled=False)

# Invalid - would raise TypeError
# config = configure_database("localhost", 5432, False)
```

**Positional-Only Arguments (Python 3.8+):**
Use `/` to indicate parameters that must be positional (cannot be passed as keywords). Useful for parameters where names have no semantic meaning or to preserve API flexibility.

```python
def calculate_power(base: float, exponent: float, /) -> float:
    """
    base and exponent must be passed positionally.
    This allows renaming them later without breaking client code.
    """
    return base ** exponent

calculate_power(2, 3)      # Valid
# calculate_power(base=2, exponent=3)  # TypeError: positional-only argument
```

### Combining All Parameter Types

The complete order of parameters is:
1.  Positional-only (`/` before these)
2.  Regular positional-or-keyword
3.  Variable positional (`*args`)
4.  Keyword-only (`*` before these)
5.  Variable keyword (`**kwargs`)

```python
def complex_function(
    pos_only: int,          # 1. Positional only (if / followed)
    standard: str,          # 2. Standard
    /,
    pos_or_kw: float,       # 3. Can be either (before / is pos-only, after is standard)
    *args: int,             # 4. Extra positional
    kw_only: bool,          # 5. Must be keyword
    **kwargs: str           # 6. Extra keywords
) -> None:
    pass
```

## 5.3 Scope and Namespaces

Understanding where variables exist and are accessible is crucial for avoiding bugs. Python uses the **LEGB rule** for scope resolution.

### The LEGB Rule

When Python encounters a name (variable), it searches in this order:

1.  **L**ocal: The current function's scope.
2.  **E**nclosing: The scope of any enclosing (outer) functions.
3.  **G**lobal: The module-level scope.
4.  **B**uilt-in: Python's built-in names (`len`, `print`, etc.).

```python
# Global scope
x: int = "global x"

def outer_function() -> None:
    # Enclosing scope
    x: int = "enclosing x"
    
    def inner_function() -> None:
        # Local scope
        x: int = "local x"
        print(f"Inner: {x}")    # Finds local x
    
    inner_function()
    print(f"Outer: {x}")        # Finds enclosing x

outer_function()
print(f"Global: {x}")           # Finds global x

# Output:
# Inner: local x
# Outer: enclosing x
# Global: global x
```

### The `global` and `nonlocal` Keywords

**`global`**: Declares that a variable refers to the global scope.

```python
counter: int = 0

def increment() -> None:
    global counter  # Without this: UnboundLocalError
    counter += 1

increment()
print(counter)  # 1
```

**Industry Warning:** Avoid `global` in production code. It creates hidden dependencies and makes testing difficult. Instead, use classes or pass state as parameters.

**`nonlocal`**: Declares that a variable refers to the nearest enclosing scope (not global, not local).

```python
def make_counter() -> callable:
    count: int = 0  # Enclosing scope
    
    def increment() -> int:
        nonlocal count  # Refers to count in make_counter's scope
        count += 1
        return count
    
    return increment

counter_a = make_counter()
print(counter_a())  # 1
print(counter_a())  # 2

counter_b = make_counter()  # Independent counter
print(counter_b())  # 1
```

## 5.4 First-Class Functions and Higher-Order Functions

In Python, functions are **first-class citizens**: they can be passed as arguments, returned from other functions, and assigned to variables.

### Functions as Arguments

```python
from typing import Callable

def apply_operation(
    values: list[int], 
    operation: Callable[[int], int]
) -> list[int]:
    """Apply a function to each item in a list."""
    return [operation(x) for x in values]

def square(x: int) -> int:
    return x ** 2

def double(x: int) -> int:
    return x * 2

numbers = [1, 2, 3, 4]
print(apply_operation(numbers, square))   # [1, 4, 9, 16]
print(apply_operation(numbers, double))   # [2, 4, 6, 8]
```

### Functions as Return Values (Closures)

A closure is a function that remembers the environment in which it was created.

```python
from typing import Callable

def make_multiplier(factor: int) -> Callable[[int], int]:
    """Create a function that multiplies its input by factor."""
    def multiply(x: int) -> int:
        return x * factor  # factor is "remembered" from enclosing scope
    return multiply

double = make_multiplier(2)
triple = make_multiplier(3)

print(double(5))  # 10
print(triple(5))  # 15
```

**Late Binding Trap:** A common closure pitfall occurs with loops.

```python
def create_multipliers() -> list[Callable[[], int]]:
    """WARNING: This has a bug due to late binding!"""
    return [lambda: i * 2 for i in range(5)]

multipliers = create_multipliers()
for m in multipliers:
    print(m())  # Output: 8, 8, 8, 8, 8 (not 0, 2, 4, 6, 8!)

# FIX: Bind i as a default argument (evaluated at definition time)
def create_multipliers_fixed() -> list[Callable[[], int]]:
    return [lambda i=i: i * 2 for i in range(5)]  # i=i captures current value
```

## 5.5 Lambda Expressions

Lambda functions are small, anonymous functions restricted to a single expression.

```python
# Syntax: lambda parameters: expression
square = lambda x: x ** 2
print(square(5))  # 25

# Common use: sorting with custom key
users = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25},
    {"name": "Charlie", "age": 35}
]

# Sort by age
sorted_users = sorted(users, key=lambda u: u["age"])

# Used with higher-order functions
numbers = [1, 2, 3, 4, 5, 6]
evens = list(filter(lambda x: x % 2 == 0, numbers))
```

**Industry Guidelines:**
*   Prefer regular `def` functions for complex logic—lambdas should be simple and immediately obvious.
*   For sorting and mapping, often operator module functions or attrgetter/itemgetter are clearer than lambdas:
    ```python
    from operator import itemgetter
    sorted_users = sorted(users, key=itemgetter("age"))
    ```

## 5.6 Modules and Packages

As programs grow, splitting code into multiple files (modules) and directories (packages) becomes essential.

### Import Styles

**Absolute Imports (Preferred):**
```python
# File structure: project/utils/helpers.py
# Importing from: project/main.py
from utils.helpers import format_date
import utils.helpers  # Access as utils.helpers.format_date()
```

**Relative Imports:**
Used within packages, relative to the current module.
```python
from . import sibling_module      # Same package
from .. import parent_module      # Parent package
from .subpackage import module    # Subdirectory
```
*Note: Relative imports can be fragile during refactoring. Absolute imports are more explicit and preferred.*

### The `if __name__ == "__main__"` Idiom

When a Python file is run directly, `__name__` is set to `"__main__"`. When imported as a module, `__name__` is the module name. This allows files to be both reusable modules and executable scripts.

```python
# calculator.py
def add(a: float, b: float) -> float:
    return a + b

def main() -> None:
    """Entry point when run as script."""
    print("Running calculator...")
    result = add(2, 3)
    print(f"2 + 3 = {result}")

if __name__ == "__main__":
    main()
```

**Industry Standard:** Every executable script should use this pattern. It prevents code from running during imports and enables testing of the `main()` function.

### Package Structure

A package is a directory containing an `__init__.py` file (can be empty in Python 3.3+, but still useful for initialization).

```
my_project/
├── README.md
├── requirements.txt
├── setup.py
└── src/
    └── mypackage/
        ├── __init__.py          # Package initialization
        ├── core.py              # Module
        ├── utils/
        │   ├── __init__.py
        │   ├── validators.py
        │   └── formatters.py
        └── cli.py               # Command-line interface
```

**Modern Python (3.3+):** Implicit namespace packages allow splitting packages across directories, but explicit `__init__.py` files remain best practice for clarity.

### Import Best Practices

1.  **Order imports**: Standard library, third-party, local application (separated by blank lines).
    ```python
    import os
    import sys
    from datetime import datetime

    import requests
    from pydantic import BaseModel

    from mypackage.utils import helper
    ```

2.  **Avoid circular imports**: If module A imports B, and B imports A, you have a circular dependency. Solutions:
    *   Refactor to remove the dependency
    *   Use lazy imports (import inside function)
    *   Use type hint imports with `TYPE_CHECKING` flag

3.  **Avoid `from module import *`**: It pollutes the namespace and hides dependencies. Explicit is better than implicit.

## Summary

Functions are the building blocks of abstraction in Python. You have learned to define them with **type hints** and **docstrings**, handle flexible argument patterns with `*args` and `**kwargs`, and avoid the mutable default argument trap. You understand Python's **LEGB scope resolution** and can use `nonlocal` for closures that maintain state.

We explored **first-class functions**, enabling functional programming patterns like passing functions as arguments and creating closures. While **lambda expressions** offer conciseness for simple operations, you recognize when a named function improves readability. Finally, you can structure code into **modules and packages**, using absolute imports and the `if __name__ == "__main__"` idiom to create maintainable, professional-grade projects.

However, as programs grow more complex, organizing data and behavior together becomes necessary. Simple functions operating on dictionaries become unwieldy when dealing with complex entities that have both state and behavior. In the next chapter, we will explore Python's object-oriented programming model, which allows you to encapsulate data and the operations that manipulate that data into cohesive, reusable structures.

**Next Chapter**: Chapter 6: Classes and Objects.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='4. core_data_structures.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../3. object_oriented_programming/6. classes_and_objects.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
